In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials  #bayesian optimization
from tpot import TPOTClassifier #genetic algorithms optimization
import optuna #to optimize hyperparameters of the model
import warnings
warnings.filterwarnings('ignore')


import sklearn.svm

In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df['Glucose'] = np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin'] = np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness'] = np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [4]:
x = df.drop('Outcome',axis=1)
y = df['Outcome']

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=33)

## normally

In [6]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(x_train,y_train)
y_predict = rf.predict(x_test)

In [7]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [8]:
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

[[83 16]
 [30 25]]
0.7012987012987013
              precision    recall  f1-score   support

           0       0.73      0.84      0.78        99
           1       0.61      0.45      0.52        55

    accuracy                           0.70       154
   macro avg       0.67      0.65      0.65       154
weighted avg       0.69      0.70      0.69       154



## by applying some parameters by choice

In [9]:
rfc1 = RandomForestClassifier(n_estimators=500,criterion='gini',max_features='sqrt',min_samples_leaf=10,random_state=100)
rfc1.fit(x_train,y_train)
y_predict = rfc1.predict(x_test)
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

[[87 12]
 [28 27]]
0.7402597402597403
              precision    recall  f1-score   support

           0       0.76      0.88      0.81        99
           1       0.69      0.49      0.57        55

    accuracy                           0.74       154
   macro avg       0.72      0.68      0.69       154
weighted avg       0.73      0.74      0.73       154



## RandomizedSearchCV

In [10]:
n_estimators = [int(x) for x in np.linspace(200,2000,10)]
max_features = ['sqrt','auto','log2']
max_depth = [int(x) for x in np.linspace(10,1000,10)]
min_samples_split = [1,3,4,5,7,9]
min_samples_leaf = [1,2,4,6,8]

random_grid = {'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,
              'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['sqrt', 'auto', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [11]:
rfc = RandomForestClassifier()
rs_cv = RandomizedSearchCV(estimator=rfc,param_distributions=random_grid,n_iter=100,cv=2,verbose=2,random_state=100,n_jobs=1)
rs_cv.fit(x_train,y_train)

Fitting 2 folds for each of 100 candidates, totalling 200 fits
[CV] END criterion=entropy, max_depth=670, max_features=sqrt, min_samples_leaf=8, min_samples_split=1, n_estimators=200; total time=   0.0s


C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

[CV] END criterion=entropy, max_depth=670, max_features=sqrt, min_samples_leaf=8, min_samples_split=1, n_estimators=200; total time=   0.0s
[CV] END criterion=gini, max_depth=560, max_features=log2, min_samples_leaf=1, min_samples_split=7, n_estimators=1600; total time=   3.6s
[CV] END criterion=gini, max_depth=560, max_features=log2, min_samples_leaf=1, min_samples_split=7, n_estimators=1600; total time=   3.5s
[CV] END criterion=gini, max_depth=340, max_features=auto, min_samples_leaf=2, min_samples_split=9, n_estimators=2000; total time=   4.2s
[CV] END criterion=gini, max_depth=340, max_features=auto, min_samples_leaf=2, min_samples_split=9, n_estimators=2000; total time=   4.2s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=2000; total time=   4.5s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=3, n_estimators=2000; total time=   4.5s
[CV] END criterion=gini, max

C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

[CV] END criterion=gini, max_depth=120, max_features=sqrt, min_samples_leaf=6, min_samples_split=1, n_estimators=600; total time=   0.3s


...

[CV] END criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=1, n_estimators=2000; total time=   1.2s


C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\AKSHY\Anaconda3\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

[CV] END criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=1, n_estimators=2000; total time=   1.5s
[CV] END criterion=gini, max_depth=1000, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=1600; total time=   4.9s
[CV] END criterion=gini, max_depth=1000, max_features=sqrt, min_samples_leaf=2, min_samples_split=4, n_estimators=1600; total time=   4.6s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=4, min_samples_split=4, n_estimators=200; total time=   0.5s
[CV] END criterion=entropy, max_depth=120, max_features=sqrt, min_samples_leaf=4, min_samples_split=4, n_estimators=200; total time=   0.4s
[CV] END criterion=entropy, max_depth=890, max_features=log2, min_samples_leaf=6, min_samples_split=7, n_estimators=600; total time=   1.7s
[CV] END criterion=entropy, max_depth=890, max_features=log2, min_samples_leaf=6, min_samples_split=7, n_estimators=600; total time=   1.7s
[CV] END criterion=gin

C:\Users\AKSHY\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.7752443  0.77035831 0.77361564 0.76547231 0.77361564
 0.77035831        nan 0.77850163 0.7752443         nan 0.76547231
 0.78013029 0.77035831 0.77035831 0.7752443  0.77361564 0.76547231
        nan        nan 0.76872964 0.77850163 0.76547231 0.77687296
        nan        nan 0.77198697 0.7752443  0.77035831 0.7752443
 0.77198697        nan 0.77198697 0.76872964        nan 0.77361564
 0.77198697 0.77035831 0.77850163 0.76384365 0.77361564 0.77361564
        nan 0.78013029 0.77035831 0.77850163 0.76710098 0.77198697
 0.76872964 0.7752443         nan        nan        nan 0.77687296
 0.77361564 0.7752443  0.76872964 0.78175896 0.78013029 0.77035831
 0.77035831 0.77361564 0.77035831 0.77850163        nan        nan
 0.77035831 0.77687296 0.77198697 0.77198697 0.77035831 0.78013029
 0.77198697 0.76710098 0.77361564 0.76547231 0.7622149

RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['sqrt', 'auto',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 3, 4, 5, 7, 9],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [12]:
rs_cv.best_params_

{'n_estimators': 1600,
 'min_samples_split': 7,
 'min_samples_leaf': 4,
 'max_features': 'log2',
 'max_depth': 120,
 'criterion': 'entropy'}

In [13]:
rs_cv

RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['sqrt', 'auto',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 3, 4, 5, 7, 9],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [14]:
rs_cv.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=120, max_features='log2',
                       min_samples_leaf=4, min_samples_split=7,
                       n_estimators=1600)

In [15]:
best_random_grid = rs_cv.best_estimator_
y_predict = best_random_grid.predict(x_test)
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

[[85 14]
 [24 31]]
0.7532467532467533
              precision    recall  f1-score   support

           0       0.78      0.86      0.82        99
           1       0.69      0.56      0.62        55

    accuracy                           0.75       154
   macro avg       0.73      0.71      0.72       154
weighted avg       0.75      0.75      0.75       154



## GridSearchCV

In [16]:
rs_cv.best_params_

{'n_estimators': 1600,
 'min_samples_split': 7,
 'min_samples_leaf': 4,
 'max_features': 'log2',
 'max_depth': 120,
 'criterion': 'entropy'}

In [17]:
para_grid = {'criterion':[rs_cv.best_params_['criterion']],'max_depth':[rs_cv.best_params_['max_depth']],
             'max_features':[rs_cv.best_params_['max_features']],
            'min_samples_leaf':[rs_cv.best_params_['min_samples_leaf'],
                                rs_cv.best_params_['min_samples_leaf']+2,
                                rs_cv.best_params_['min_samples_leaf']+4],
            'min_samples_split':[rs_cv.best_params_['min_samples_split']-1,
                                rs_cv.best_params_['min_samples_split']-2,
                                rs_cv.best_params_['min_samples_split'],
                                rs_cv.best_params_['min_samples_split']+1,
                                rs_cv.best_params_['min_samples_split']+2],
            'n_estimators':[rs_cv.best_params_['n_estimators']-200,
                               rs_cv.best_params_['n_estimators']-100,
                               rs_cv.best_params_['n_estimators'],
                               rs_cv.best_params_['n_estimators']+100,
                               rs_cv.best_params_['n_estimators']+200]}
print(para_grid)

{'criterion': ['entropy'], 'max_depth': [120], 'max_features': ['log2'], 'min_samples_leaf': [4, 6, 8], 'min_samples_split': [6, 5, 7, 8, 9], 'n_estimators': [1400, 1500, 1600, 1700, 1800]}


In [18]:
1*1*1*3*5*5

75

In [19]:
rfc = RandomForestClassifier()
gs_cv = GridSearchCV(estimator=rfc,param_grid=para_grid,cv=10,n_jobs=-1,verbose=2)
gs_cv.fit(x_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [120],
                         'max_features': ['log2'],
                         'min_samples_leaf': [4, 6, 8],
                         'min_samples_split': [6, 5, 7, 8, 9],
                         'n_estimators': [1400, 1500, 1600, 1700, 1800]},
             verbose=2)

In [20]:
1*1*1*2*3*3

18

In [21]:
gs_cv.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=120, max_features='log2',
                       min_samples_leaf=6, min_samples_split=9,
                       n_estimators=1400)

In [22]:
best_grid = gs_cv.best_estimator_

In [23]:
y_predict = best_grid.predict(x_test)
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

[[86 13]
 [25 30]]
0.7532467532467533
              precision    recall  f1-score   support

           0       0.77      0.87      0.82        99
           1       0.70      0.55      0.61        55

    accuracy                           0.75       154
   macro avg       0.74      0.71      0.72       154
weighted avg       0.75      0.75      0.75       154



# Automated Hyperparameter Tuning

## Bayesian Optimization

In [35]:
space = {'criterion':hp.choice('criterion',['entropy','gini']),
        'max_depth':hp.quniform('max_depth',10,1200,10),
        'max_features':hp.choice('max_features',['auto','sqrt','log2',None]),
        'min_samples_leaf':hp.uniform('min_samples_leaf',0,0.5),
        'min_samples_split':hp.uniform('min_samples_split',0,1),
        'n_estimators':hp.choice('n_estimators',[10,50,300,750,1200,1300])}
space

{'criterion': <hyperopt.pyll.base.Apply at 0x295328474f0>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x295328474c0>,
 'max_features': <hyperopt.pyll.base.Apply at 0x29532847520>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x295334d5c70>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x295334d5b50>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x29532eeeac0>}

In [36]:
space['max_features']

In [39]:
def objective(space):
    model = RandomForestClassifier(criterion=space['criterion'],max_depth=space['max_depth'],
                                  max_features=space['max_features'],min_samples_leaf=space['min_samples_leaf'],
                                  min_samples_split=space['min_samples_split'],n_estimators=space['n_estimators'])
    accuracy = cross_val_score(model,x_train,y_train,cv=5).mean()
    return {'loss': -accuracy,'status': STATUS_OK}

In [40]:
trials = Trials()
best = fmin(fn=objective,space= space,algo=tpe.suggest,max_evals=80,trials=trials)
best

100%|███████████████████████████████████████████████| 80/80 [10:47<00:00,  8.09s/trial, best loss: -0.7768759163001466]


{'criterion': 1,
 'max_depth': 230.0,
 'max_features': 0,
 'min_samples_leaf': 0.017795696748113966,
 'min_samples_split': 0.0035085856671450274,
 'n_estimators': 2}

In [43]:
crit = {0:'entropy',1:'gini'}
feat = {0:'auto', 1:'sqrt', 2:'log2', 3:None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200, 5: 1300, 6: 1500}

In [44]:
print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
auto
300


In [45]:
best['min_samples_leaf']

0.017795696748113966

In [53]:
trained_forest = RandomForestClassifier(criterion=crit[best['criterion']],max_depth=best['max_depth'],
                                       max_features=feat[best['max_features']],min_samples_leaf=best['min_samples_leaf'],
                                       min_samples_split=best['min_samples_split'],
                                       n_estimators = est[best['n_estimators']])
trained_forest.fit(x_train,y_train)
y_predict = trained_forest.predict(x_test)
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

[[88 11]
 [29 26]]
0.7402597402597403
              precision    recall  f1-score   support

           0       0.75      0.89      0.81        99
           1       0.70      0.47      0.57        55

    accuracy                           0.74       154
   macro avg       0.73      0.68      0.69       154
weighted avg       0.73      0.74      0.73       154



## Genetic Algorithms

In [62]:
n_estimators = [int(x) for x in np.linspace(200,2000,10)]
max_features = ['sqrt','auto','log2']
max_depth = [int(x) for x in np.linspace(10,1000,10)]
min_samples_split = [1,3,4,5,7,9]
min_samples_leaf = [1,2,4,6,8]

random_grid = {'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,
              'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['sqrt', 'auto', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [63]:
tpc = TPOTClassifier(generations=5,population_size=24,offspring_size=12,verbosity=2,early_stop=12,
                    config_dict={'sklearn.ensemble.RandomForestClassifier':random_grid},cv=4,scoring='accuracy')
tpc.fit(x_train,y_train)
y_predict = tpc.predict(x_test)
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.783454290807232

Generation 2 - Current best internal CV score: 0.783454290807232

Generation 3 - Current best internal CV score: 0.783454290807232

Generation 4 - Current best internal CV score: 0.783454290807232

Generation 5 - Current best internal CV score: 0.783454290807232

Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=gini, max_depth=450, max_features=log2, min_samples_leaf=6, min_samples_split=9, n_estimators=1400)
[[87 12]
 [24 31]]
0.7662337662337663
              precision    recall  f1-score   support

           0       0.78      0.88      0.83        99
           1       0.72      0.56      0.63        55

    accuracy                           0.77       154
   macro avg       0.75      0.72      0.73       154
weighted avg       0.76      0.77      0.76       154



## Optuna

In [81]:
def objective(trial):
    classifier = trial.suggest_categorical('classifier',['RandoForest','SVC'])
    
    if classifier == "RandomForest":
        n_estimators = trial.suggest_int(n_estimators,200,2000,10)
        max_depth = int(trial.suggest_float('max_depth',10,100,log=True))
        
        clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_C',1e-10,1e10,log=True)
        clf = sklearn.svm.SVC(C=c, gamma='auto')
        
    return sklearn.model_selection.cross_val_score(clf,x_train,y_train,n_jobs=-1,cv=3).mean()

In [82]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

trial = study.best_trial

print('Accuracy:',trial.value)
print('Best Hyperparameters:',trial.params)

[I 2021-10-12 18:45:28,813] A new study created in memory with name: no-name-7f56e435-befa-42f1-9e37-19902fd48aa5
[I 2021-10-12 18:45:28,896] Trial 0 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 1.7929960014216984e-08}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:28,991] Trial 1 finished with value: 0.6530926191614858 and parameters: {'classifier': 'RandoForest', 'svc_C': 0.5949261016862427}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:29,065] Trial 2 finished with value: 0.6530926191614858 and parameters: {'classifier': 'RandoForest', 'svc_C': 3.3178984551321806e-07}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:29,135] Trial 3 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 2.2957299495622588e-09}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:29,231] Trial 4 finished with value: 0.6530926191614858 and parameters

[I 2021-10-12 18:45:32,381] Trial 39 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 3.1253260704101304e-08}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:32,458] Trial 40 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 3.415090478255078e-05}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:32,543] Trial 41 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 9.572705037482295e-10}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:32,623] Trial 42 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 5.33035022140902e-08}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:32,701] Trial 43 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 7.450779852288183e-06}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:

[I 2021-10-12 18:45:36,000] Trial 78 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 0.06299447265923837}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:36,107] Trial 79 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 0.40699626057680205}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:36,187] Trial 80 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 0.0002551884040762923}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:36,289] Trial 81 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 1.2273852994665901}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:36,391] Trial 82 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_C': 5.266258853004559}. Best is trial 0 with value: 0.6530926191614858.
[I 2021-10-12 18:45:36,503] Tri

Accuracy: 0.6530926191614858
Best Hyperparameters: {'classifier': 'SVC', 'svc_C': 1.7929960014216984e-08}


In [83]:
trial

FrozenTrial(number=0, values=[0.6530926191614858], datetime_start=datetime.datetime(2021, 10, 12, 18, 45, 28, 817705), datetime_complete=datetime.datetime(2021, 10, 12, 18, 45, 28, 896636), params={'classifier': 'SVC', 'svc_C': 1.7929960014216984e-08}, distributions={'classifier': CategoricalDistribution(choices=('RandoForest', 'SVC')), 'svc_C': LogUniformDistribution(high=10000000000.0, low=1e-10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)

In [84]:
study.best_params

{'classifier': 'SVC', 'svc_C': 1.7929960014216984e-08}

In [88]:
# take following parameters accordingly above results
rfc=RandomForestClassifier(n_estimators=330,max_depth=330)
rfc.fit(x_train,y_train)
y_predict = rfc.predict(x_test)
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

[[85 14]
 [24 31]]
0.7532467532467533
              precision    recall  f1-score   support

           0       0.78      0.86      0.82        99
           1       0.69      0.56      0.62        55

    accuracy                           0.75       154
   macro avg       0.73      0.71      0.72       154
weighted avg       0.75      0.75      0.75       154

